# Strat discovery
Functionality:
- Review constants
- Run
- Allow saving progress and quitting
- End with visualizations and choosing which strats to go

## Imports

In [1]:
import discovery_downloader as ddl
import adf_calculator as adfc
# from datetime import datetime
# import numpy as np
# import time

## Constants and Paramaters

In [10]:
RESUME = True #If false, then restart, else use current progress

In [2]:
## File names
PATHNAME = "../data/strat-discovery/"
HOURPATH = "../data/hour/"
MINUTEPATH = "../data/minute/"
ALL_ISOLATED_PAIRS = "all_isolated_pairs.csv"
ADF_STATS = "adf_stats.csv"

## Get all margin assets on Binance

In [12]:
ddl.download_all_available_margin_assets(PATHNAME, ALL_ISOLATED_PAIRS) #RUNTIME: 1 second

## Download hourly data

In [3]:
ddl.download_all_hourly_data(PATHNAME, ALL_ISOLATED_PAIRS, HOURPATH)

Operation took 27.74 minutes to download 231 files


## Compute ADF tests 

In [ ]:
adfc.compute_adf_tests(PATHNAME, HOURPATH, ALL_ISOLATED_PAIRS, ADF_STATS, 10000)

## Strat searching